In [ ]:
!pip install langchain faiss-cpu sentence-transformers chromadb

In [32]:
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.0/284.0 kB 6.3 MB/s eta 0:00:00


In [53]:
from langchain.llms import HuggingFaceHub
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import CharacterTextSplitter,RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain.vectorstores import FAISS, Chroma
from langchain.chains import RetrievalQA, LLMChain
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain import HuggingFaceHub


In [ ]:
from google.colab import userdata
HF_TOKEN = userdata.get('HFToken')
#print(HF_TOKEN)
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HF_TOKEN

## External data/document - ETL

In [ ]:
import nest_asyncio

nest_asyncio.apply()

In [ ]:
#WEBSITE_URL = "https://www.ers.usda.gov/data-products/food-access-research-atlas/go-to-the-atlas.aspx"

In [ ]:
#loader = WebBaseLoader(WEBSITE_URL)
#loader.requests_per_second = 1
#docs = loader.aload()

Fetching pages: 100%|##########| 1/1 [00:00<00:00, 15.21it/s]


In [33]:
loader = PyPDFDirectoryLoader("/usr/src/data/")
docs = loader.load()

In [ ]:
docs

## Text Splitting - Chunking

In [35]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=256, chunk_overlap=20)
chunks = text_splitter.split_documents(docs)

In [36]:
chunks[0]

Document(page_content='To our shareowners:\nOne thing we’ve learned from the COVID-19 crisis is how important Amazon has become to our customers. We', metadata={'source': '/usr/src/data/2019-Shareholder-Letter.pdf', 'page': 0})

In [37]:
chunks[1]

Document(page_content='want you to know we take this responsibility seriously, and we’re proud of the work our teams are doing to helpcustomers through this difficult time.', metadata={'source': '/usr/src/data/2019-Shareholder-Letter.pdf', 'page': 0})

In [38]:
chunks[2]

Document(page_content='Amazonians are working around the clock to get necessary supplies delivered directly to the doorsteps of people', metadata={'source': '/usr/src/data/2019-Shareholder-Letter.pdf', 'page': 0})

In [39]:
chunks[3]

Document(page_content='who need them. The demand we are seeing for essential products has been and remains high. But unlike apredictable holiday surge, this spike occurred with little warning, creating major challenges for our suppliers anddelivery network. We quickly', metadata={'source': '/usr/src/data/2019-Shareholder-Letter.pdf', 'page': 0})

In [40]:
chunks[4]

Document(page_content='network. We quickly prioritized the stocking and delivery of essential household staples, medicalsupplies, and other critical products.', metadata={'source': '/usr/src/data/2019-Shareholder-Letter.pdf', 'page': 0})

## Embeddings

In [41]:
embeddings = HuggingFaceInferenceAPIEmbeddings(
    api_key=HF_TOKEN, model_name="BAAI/bge-base-en-v1.5"
)

## Vector Store - FAISS or ChromaDB

In [42]:
vectorstore = Chroma.from_documents(chunks, embeddings)

In [43]:
vectorstore

In [44]:
query = "What is Amazon's doing in the field of generative AI?"
search = vectorstore.similarity_search(query)

In [45]:
search[0].page_content

'For now, my own time and thinking continues to be focused on COVID-19 and how Amazon can help while'

## Retriever

In [46]:
retriever = vectorstore.as_retriever(
    search_type="mmr", #similarity
    search_kwargs={'k': 4}
)

In [47]:
retriever.get_relevant_documents(query)

[Document(page_content='For now, my own time and thinking continues to be focused on COVID-19 and how Amazon can help while', metadata={'page': 4, 'source': '/usr/src/data/2019-Shareholder-Letter.pdf'}),
 Document(page_content='removed over half a million offers from our stores due to COVID-based price gouging, and we’ve suspendedmore than 6,000 selling accounts globally for violating our fair-pricing policies. Amazon turned over informationabout sellers we suspect engaged in', metadata={'page': 1, 'source': '/usr/src/data/2019-Shareholder-Letter.pdf'}),
 Document(page_content='We are taking steps to help those most vulnerable to the virus, setting aside the first hour of shopping at WholeFoods each day for seniors. We have temporarily closed Amazon Books, Amazon 4-star, and Amazon Pop Upstores because they don’t sell essential', metadata={'page': 0, 'source': '/usr/src/data/2019-Shareholder-Letter.pdf'}),
 Document(page_content='video service toaccommodate a 4,000% spike in demand to 

## Large Language Model - Open Source

In [68]:
llm = HuggingFaceHub(
   repo_id="huggingfaceh4/zephyr-7b-alpha",
  model_kwargs={"temperature": 0.5, "max_length": 64,"max_new_tokens":512}
)

In [54]:
# llm = HuggingFaceHub(
#     repo_id="Xenova/gpt-3.5-turbo",
 #    model_kwargs={"temperature": 0.5, "max_length": 64,"max_new_tokens":512},
#     task="text-generation"
# )

In [ ]:
# llm = HuggingFaceHub(
#     repo_id="google/flan-t5-xl",
 #    model_kwargs={"temperature": 0.5, "max_length": 64,"max_new_tokens":512},
#     task="text-generation"
# )

## Prompt Template and User Input

In [62]:
query = "What is Amazon Covid policy?"

prompt = f"""
 <|system|>
You are an AI assistant that follows instruction extremely well.
Please be truthful and give direct answers
</s>
 <|user|>
 {query}
 </s>
 <|assistant|>
"""

## RAG RetrievalQA chain

In [69]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="refine", retriever=retriever)

In [59]:
# print(qa) testing

combine_documents_chain=RefineDocumentsChain(initial_llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['context_str', 'question'], template='Context information is below. \n------------\n{context_str}\n------------\nGiven the context information and not prior knowledge, answer the question: {question}\n'), llm=HuggingFaceHub(client=<InferenceClient(model='Xenova/gpt-3.5-turbo', timeout=None)>, repo_id='Xenova/gpt-3.5-turbo', task='text-generation', model_kwargs={'temperature': 0.5, 'max_length': 64, 'max_new_tokens': 512})), refine_llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['context_str', 'existing_answer', 'question'], template="The original question is as follows: {question}\nWe have provided an existing answer: {existing_answer}\nWe have the opportunity to refine the existing answer (only if needed) with some more context below.\n------------\n{context_str}\n------------\nGiven the new context, refine the original answer to better answer the question. If the c

In [70]:
# prompt: generate code for response using variable "qa"

response = qa.run(prompt)


In [71]:
response

"The original question is as follows: \n <|system|>\nYou are an AI assistant that follows instruction extremely well.\nPlease be truthful and give direct answers\n</s>\n <|user|>\n What is Amazon Covid policy?\n </s>\n <|assistant|>\n\nWe have provided an existing answer: The original question is as follows: \n <|system|>\nYou are an AI assistant that follows instruction extremely well.\nPlease be truthful and give direct answers\n</s>\n <|user|>\n What is Amazon Covid policy?\n </s>\n <|assistant|>\n\nWe have provided an existing answer: The original question is as follows: \n <|system|>\nYou are an AI assistant that follows instruction extremely well.\nPlease be truthful and give direct answers\n</s>\n <|user|>\n What is Amazon Covid policy?\n </s>\n <|assistant|>\n\nWe have provided an existing answer: Context information is below. \n------------\nFor now, my own time and thinking continues to be focused on COVID-19 and how Amazon can help while\n------------\nGiven the context info

## Chain

In [72]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain.prompts import ChatPromptTemplate

In [73]:
template = """
 <|system|>
You are an AI assistant that follows instruction extremely well.
Please be truthful and give direct answers
</s>
 <|user|>
 {query}
 </s>
 <|assistant|>
"""

In [74]:
prompt = ChatPromptTemplate.from_template(template)

In [75]:
rag_chain = (
    {"context": retriever,  "query": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [77]:
response = rag_chain.invoke("What is Amazon's doing in the field of generative AI ?")

In [78]:
print(response)

Human: 
 <|system|>
You are an AI assistant that follows instruction extremely well.
Please be truthful and give direct answers
</s>
 <|user|>
 What is Amazon's doing in the field of generative AI ?
 </s>
 <|assistant|>
Amazon is actively investing in and developing generative AI technologies through its Amazon Web Services (AWS) division. AWS offers a range of generative AI services, including Amazon Textract, which can automatically extract text and data from documents, and Amazon Transcribe, which can transcribe audio into text. Additionally, AWS offers Amazon SageMaker, a machine learning platform that can be used to train and deploy generative AI models. Amazon is also working on developing generative AI models for
